In [49]:
import numpy as np

In [50]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Step 0 : Load the Data

In [111]:
x = np.load('x.npy') # 5000 images
y = np.load('y.npy')

theta_0 = np.load('theta_0.npy')
theta_1 = np.load('theta_1.npy')

## Step 1. Convert Pixels Using Fixed Point Notation

In [112]:
# image [0.0, 1.0[
# 4 bits [0 signed, 0 integer, 4 fractional]
def float_to_fix_point_unsigned(x: float, n: int) -> int:
    if x >= 1.0:
        x = 0.99999
    res = int(x * pow(2, n))
    return res

imgIndex = 0

image = x[imgIndex]
xp = np.hstack((1, image))

image_fp04 = np.zeros_like(xp)
for i in range(xp.shape[0]):
    image_fp04[i] = float_to_fix_point_unsigned(xp[i], 4)

## Step 2. Convert First Hidden Layer Weights Into Fixed Point Notation

In [ ]:
# theta_0 [-1.463369, 1.00899]
# 4 bits [1 signed, 1 integer, 2 fractional]
def weight_to_fixed_point_4bits(weight: float):
    binary = ""
    # Signed bit
    if weight <= -0.25:
        binary += "1"
    else:
        binary += "0"
    
    # Integer part
    integer_part = int(weight)
    binary_conversion = {0: "0", 1 :"1"}
    binary += binary_conversion[abs(integer_part)]

    # Fractional part
    fractional_part = abs(weight - integer_part)
    if fractional_part >= 0.5:
        binary += "1"
        fractional_part -= 0.5
    else:
        binary += "0"
    
    if fractional_part >= 0.25:
        binary += "1"
    else:
        binary += "0"
    # if int(binary, 2) > 0:
    #    print(f"weights {round(weight, 2)} binary {binary}")
    return int(binary, 2)

def fixed_point_to_float_4bits(weight: int):
    binary = bin(weight)
    res = 0.0
    if binary[0]


theta_0_fp = np.zeros_like(theta_0)
for i in range(theta_0.shape[0]): # 25
    for j in range(theta_0.shape[1]): # 401
        theta_0_fp[i][j] = weight_to_fixed_point(theta_0[i][j])

## Step 3. Make The Second Hidden Layer Multiplication X*W 

In [120]:
# We want to simulate this result
# hiddenLayer1 = np.dot(xp,theta_0.T)
fl_hiddenLayer1 = []
for i in range(0, len(theta_0_fp)): # [0, 24]
    row = theta_0_fp[i]
    product = 0
    for j in range(0, len(row)): # [0, 400]
        weight_fp = row[j]
        pixel = image_fp04[j]
        print(bin(int(pixel)))
        # todo convertir les valeurs en nombres de 4 bits signés?
        # sproduct = product + weight * pixel
    fl_hiddenLayer1.append(product)

# Step 4. Apply the Sigmoid Function For First Hidden Layer

# Step 5. Convert Second Hidden Layer Weights Into Fixed Point Notation

# Step 6. Make The Second Hidden Layer Multiplication X*W 

# Step 7. Apply the Sigmoid Function For Second Hidden Layer

# Step 8. Display FPGA Approximation

0b1111
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b1
0b100
0b100
0b100
0b100
0b11
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b1
0b110
0b1101
0b1111
0b1111
0b1110
0b1111
0b1111
0b111
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b1
0b1010
0b1111
0b1101
0b1000
0b101
0b100
0b1010
0b1111
0b1110
0b1
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b10
0b1100
0b1111
0b111
0b1
0b0
0b0
0b0
0b10
0b1110
0b1111
0b10
0b0
0b0
0b0
0b0
0b0
0b0
0b0
0b100
0b1110
0b1111
0b1001
0b0
0b0
0b0
0b0
0b0
0b10
0b1110
0b1110
0b1
0b0
0b0
0b0
0b0
0b0
0b0
0b10
0b1100
0b1111
0b1100
0b100
0b0
0b0
0b0
0b0
0b0
0b111
0b1111
0b1000
0b0
0b0
0b0
0b0
0b0
0b0
0b10
0b1100
0b1111
0b1100
0b10
0b0
0b0
0b0
0b0
0b0
0b0
0b1011
0b1101
0

In [ ]:
# Calculer sur le FPGA
xp = np.hstack((1, image)) # Ajouter 1 pour le biais
hiddenLayer1 = np.dot(xp,theta_0.T)

a = sigmoid(hiddenLayer1) # Sortie de la couche 1
ap = np.hstack((1, a)) # Ajouter 1 pour le biais
b = sigmoid(np.dot(ap,theta_1.T)) # Sortie de la couche 2
print("FPGA Output:",b)

# Calculer dans le notebook
pred = b.argmax()
print("Prediction:", pred, "Label:", y[idx])